In [1]:
# Verimizi düzenleyeceğimiz pandas kütüphanemizi ekliyoruz.
import pandas as pd
# Verimizin doğruluk oranına bakacağımız accuracy_scoreve kütüphanemizi ekliyoruz.
# Eğiteceğimiz verileri Train-Test olarak ayıracağımız sklearn kütüphanemizi ekliyoruz.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# XAI algoritmalarımızdan interpret algoritmamızı ekliyoruz.
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show


Verimizi " df " değişkenine atıyoruz

In [2]:
df = pd.read_csv("veriler/Iris.csv")
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


Train ve Test verilerimizi ayırıyoruz.

In [3]:
X = df.drop(["Species","Id"], axis=1)
Y = df["Species"]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, shuffle=True)

Modelimizi eğitiyoruz.

In [4]:
model = ExplainableBoostingClassifier()
model.fit(x_train, y_train)

C:\Users\umut6\anaconda4\lib\site-packages\interpret\glassbox\ebm\ebm.py:568: UserWarning: Detected multiclass problem. Forcing interactions to 0. Multiclass interactions work except for global visualizations, so the line below setting interactions to zero can be disabled if you know what you are doing.
  warn("Detected multiclass problem. Forcing interactions to 0. Multiclass interactions work except for global visualizations, so the line below setting interactions to zero can be disabled if you know what you are doing.")


ExplainableBoostingClassifier()

ExplainableBoostingClassifier() yöntemi, bir sınıflandırma modelinin performansını iyileştirmeyi amaçlayan bir yöntemdir. Bu yöntem, veri kümesine uygulanan bir dizi adım sırasında bir sınıflandırma modeli oluşturur ve bu modelin tahminlerini sürekli olarak iyileştirir. Bu yöntem aynı zamanda modelin tahminlerini açıklamak için güçlü bir özet sunar, bu sayede kullanıcılar modelin nasıl çalıştığını anlamaya yardımcı olur. Bu yöntem, veri kümesinin büyüklüğüne ve özelliklerine göre önemli ölçüde performans iyileştirmesi sağlayabilir.

Modelimizin doğruluğuna bakıyoruz.

In [5]:
y_predicted = model.predict(x_test)
accuracy = accuracy_score(y_test, y_predicted)
print("Doğruluk Oranı: ",  accuracy*100, "%")

Doğruluk Oranı:  94.66666666666667 %


Modelimizi daha rahat anlayabilmemiz için İnterpret kütüphanesinin içindeki show() fonksiyonunu kullanıyoruz.

Bu kısımda her özelliğin hangi aralıkta nasıl bir ağırlık verdiğini göstermektedir.
Hangi özelliğin daha ağır olduğunu da göstermektedir.

In [6]:
model_global = model.explain_global()
show(model_global)


<!-- http://127.0.0.1:7001/1790188505888/ -->

"Intercept" değeri, bir veri noktasının tahmini etkileyen özelliklerin ağırlık değerlerinin toplamından sonra modelin kalan tahmin yapma sürecini ifade eder. Yani, "Intercept" değeri, veri noktasının tahmini etkileyen özelliklerin ağırlık değerlerinin toplamının veri noktasının tahmini üzerindeki etkisini gösterir.

Bu açıdan, "Intercept" değeri, veri noktasının tahmini etkileyen özelliklerin ağırlık değerlerinin toplamından sonra modelin kalan tahmin yapma sürecini gösterir ve bu değer, veri noktasının tahmini üzerinde etkilidir.

Böylece, show() fonksiyonu kullanılarak, verilen veri noktası için modelin tahmin yapma sürecini anlamaya yardımcı olan bilgiler gösterilebilir ve bu bilgiler arasında "Intercept" değeri de yer almaktadır.

In [7]:
model_local = model.explain_local(x_test, y_test)
show(model_local)

<!-- http://127.0.0.1:7001/1790201402704/ -->

Bu kısımda her özelliğin ne kadar etkili olduğu yazmaktadır local olarak. Yani her test verisinde hangi özellik ne kadar etkili onu görüyoruz.

Örnek olarak 63. veride prScore (1.0) yazıyor bu %100 doğru olduğu anlamına geliyor.

Bu örneğimizde PetalLengthCm özelliğinde Iris-setosa 2.918 çıkmış. Diğer özelliklerde de Iris-setosa pozitif çıkmıştır. Bu özelliklerin ağırlıklarının hepsi toplandığında ise Irıs-setosa en yüksek çıktığı için makinenin tahmini o olmuştur.

(Kodu her çalıştırdığımızda shuffle=True olduğu için her seferinde farklı bir örnek çıkacaktır.)

In [13]:
y_test.iloc[63]

'Iris-setosa'